# Coding attention mechanism

## Attending to different parts of the input with self-Attention

### A simple self-attention mechanism without trainable weights.


In [1]:
import torch
from nbformat.v4.convert import downgrade_output

inputs = torch.tensor(
    [
        [0.43, 0.15, 0.89],  # Your     (x^1)
        [0.55, 0.87, 0.66],  # journey  (x^2)
        [0.57, 0.85, 0.64],  # starts   (x^3)
        [0.22, 0.58, 0.33],  # with     (x^4)
        [0.77, 0.25, 0.10],  # one      (x^6)
        [0.05, 0.80, 0.55],  # step     (x^7)
    ]
)

/Users/slackroo/Data_science/LLM/llm_from_scratch/.venv/lib/python3.11/site-packages/torch/nn/modules/transformer.py:20: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  device: torch.device = torch.device(torch._C._get_default_device()),  # torch.device('cpu'),


In [2]:
input_query = inputs[1]
input_query

tensor([0.5500, 0.8700, 0.6600])

In [3]:
input_1 = inputs[0]
input_1

tensor([0.4300, 0.1500, 0.8900])

In [4]:
0.55*0.43 + 0.87*0.15 + 0.66*0.89

0.9544

In [5]:

# compute the attention score

torch.dot(input_query, input_1)

tensor(0.9544)

In [6]:
res = 0.

for idx, ele in enumerate(inputs[0]):
    print(inputs[0][idx])

tensor(0.4300)
tensor(0.1500)
tensor(0.8900)


In [7]:
res = 0.
i = 0

for idx, ele in enumerate(inputs[i]):
    res += inputs[i][idx] *  input_query[idx]
res += inputs[i][idx] *  torch.dot(inputs[i], input_query)


In [8]:

inputs.shape

torch.Size([6, 3])

In [9]:
# using for all inputs
# using attn_score_2 cause we are trying to find wrt to token 2

query  = inputs[1]
attn_score_2 = torch.empty(inputs.shape[0])

for i, x_i in enumerate(inputs):
    attn_score_2[i] = torch.dot(query, x_i)

print(attn_score_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [10]:
# now from scores lets create the attention weights

# step 1 we normalize the attention scores to add up to 1

attn_weights_temp = attn_score_2/attn_score_2.sum()
print(attn_weights_temp)
print(attn_weights_temp.sum())




tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
tensor(1.0000)


In [11]:
def softmax_naive(x):
    return torch.exp(x)/torch.exp(x).sum(dim=0)

softmax_naive(attn_score_2)

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

In [12]:
attn_weights_2 = torch.softmax(attn_score_2, dim=0)  # stable and optimized stable function

print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


# Compute the context vector

![Attention Mechanism Context Vector](./docs/nb_images/cal_norm_attention_weights.png)

The context vector is a weighted sum of values, where weights come from the attention mechanism. It's computed by multiplying each input vector with its corresponding attention weight and summing the results.


In [13]:
print(attn_weights_2)

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])


In [14]:
# attn_scores
i = 1
query= inputs[i]

context_vec_2 = torch.zeros(query.shape)

for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * inputs[i]

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


# Simple self attention mechanism without trainable weights


In [15]:
# lets find attn scores for all the inputs

attn_scores = torch.empty(inputs.shape[0], inputs.shape[0])

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [16]:
attn_scores = inputs @ inputs.T
attn_weights = torch.softmax(attn_scores, dim=1)
print(attn_scores)
print(f"\nNormalised attention weights:")
print(attn_weights)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

Normalised attention weights:
tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [18]:
print(attn_weights@inputs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


# Implementing self-attention with trainable weights

## computing the attention mechanism step by step

In [21]:
inputs[0]


tensor([0.4300, 0.1500, 0.8900])

In [24]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out=2


In [27]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out))
W_key = torch.nn.Parameter(torch.rand(d_in, d_out))
W_value = torch.nn.Parameter(torch.rand(d_in, d_out))

In [28]:
query_2 = x_2 @ W_query
query_2

tensor([0.4306, 1.4551], grad_fn=<SqueezeBackward4>)

In [30]:
keys = inputs @ W_key
value = inputs @ W_value

keys.shape

torch.Size([6, 2])

In [32]:
print(keys)

tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4361, 1.1156],
        [0.2408, 0.6706],
        [0.1827, 0.3292],
        [0.3275, 0.9642]], grad_fn=<MmBackward0>)


In [33]:
# Computing attention score for the 2nd input

keys_2 = keys[1]
attn_score_22 = torch.dot(query_2, keys_2)
print(attn_score_22)

tensor(1.8524, grad_fn=<DotBackward0>)


In [35]:
attn_score_2 = query_2 @ keys.T
print(attn_score_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440],
       grad_fn=<SqueezeBackward4>)


In [38]:
# compute the normalized attenrion scores
# we add the normalizaton proposed by the original paper where we divide the scores by sqrt(d_k)

d_k = keys.shape[1]

attn_weights_2 = torch.softmax(attn_score_2 / d_k**0.5, dim=-1)
print(attn_weights_2)


tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820],
       grad_fn=<SoftmaxBackward0>)


In [39]:
attn_weights_2.sum()


tensor(1., grad_fn=<SumBackward0>)

In [40]:
# compute the context vector
# The query and key vectors are used to compute the attention weights.
# the value vector is used along with the attention weights to compute the context vector.

context_vec_2 = attn_weights_2 @ value

print(context_vec_2)


tensor([0.3061, 0.8210], grad_fn=<SqueezeBackward4>)


# 3.4.2 Implementing a compact SelfAttention class

In [44]:
import torch.nn as nn

class SelfAttentionV1O(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = torch.nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = torch.nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = torch.nn.Parameter(torch.rand(d_in , d_out))


    def forward(self, x):
        queries = inputs @ W_query
        keys = inputs @ W_key
        values = inputs @ W_value

        d_k = keys.shape[1]
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / d_k**0.5, dim=-1)
        context_vec = attn_weights @ values

        return context_vec



In [46]:
torch.manual_seed(123)

sa_v1 = SelfAttentionV1O(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [55]:
import torch.nn as nn

class SelfAttentionV2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = torch.nn.Linear(d_in, d_out, bias=qkv_bias)


    def forward(self, x):
        queries = self.W_query(inputs)
        keys = self.W_key(inputs)
        values = self.W_value(inputs)

        d_k = keys.shape[1]
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / d_k**0.5, dim=-1)
        context_vec = attn_weights @ values

        return context_vec

torch.manual_seed(789)
sa_v2 = SelfAttentionV2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


# 3.5 Hiding future words with causal attention
##  3.5.1 Applying causal attention mask



In [68]:
# your journey starts with one step

# we want generate one token at a time
# your --> want

queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
values = sa_v2.W_value(inputs)

d_k = keys.shape[1]
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / d_k**0.5, dim=-1)
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [69]:
attn_scores.shape

torch.Size([6, 6])

In [70]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [71]:
mask_simple = attn_weights * mask_simple

In [73]:
print(mask_simple.sum(dim=-1, keepdim=True))

tensor([[0.1921],
        [0.3700],
        [0.5357],
        [0.6775],
        [0.8415],
        [1.0000]], grad_fn=<SumBackward1>)


In [75]:
# it is good for optimization purposes to have the row sum to be 1

# we normalize with the row sum technique
row_sums = mask_simple.sum(dim=-1, keepdim=True)
masked_sample_norm = mask_simple / row_sums

print(masked_sample_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


In [78]:
# there is a simpler way to achieve the above in one go
# attention scores ---> masked attention scores ---> masked attention weights normalized (apply softmax)

mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)






tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


In [80]:
# torch.exp(torch.tensor(-999999999))
# or
torch.exp(torch.tensor(float("-inf")))


tensor(0.)

In [81]:
# attn_weights = torch.softmax(attn_scores / d_k**0.5, dim=-1)
attn_weights = torch.softmax(masked / d_k**0.5, dim=-1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)
